In [ ]:
# Import all the required libraries for data retrieval and visualization

import requests as req
from pprint import pprint as print
from datetime import datetime
from datetime import timedelta
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
from math import radians

In [ ]:
#All API Urls
sensors_url  = 'https://api.sealevelsensors.org/v1.0/Sensors'
properties_url = 'https://api.sealevelsensors.org/v1.0/ObservedProperties'
observations_url ='https://api.sealevelsensors.org/v1.0/Observations'
base_url_sls  = 'https://api.sealevelsensors.org/v1.0/Things'

API types include the /Sensors and /ObservedProperties, both of which are metadata, and then real time observations are updated in /Observations

In [ ]:
session = req.Session()

def get_sensor_datastreams():
    """
    Creates a list of all sensors with datastream links.
    Returns:
        sensors (list): a list of 'sensors', each sensor being a dictonary with information on the sensor
    """
    api_response = session.get(base_url_sls).json()
    def create_sensor_obj(sensor):
        """Turns a 'sensor' from the API into a succinct dictionary."""
        location = (session.get(sensor["Locations@iot.navigationLink"])).json()["value"][0]
        historical_location = (session.get(sensor["HistoricalLocations@iot.navigationLink"])).json()["value"][0]
        coordinates = location["location"]["coordinates"]
        datastreams = session.get(sensor["Datastreams@iot.navigationLink"]).json()["value"][0]
        sensor_api = session.get(datastreams["Sensor@iot.navigationLink"]).json()
        return {
          "Name":   sensor["name"],
          "ID": sensor["@iot.id"],
          "Status": sensor["properties"].get("status"),
          "Inundation Types": sensor["properties"].get("inundationTypes"),
          "Installation Type": sensor["properties"].get("installationType"),
          "Date of Last Status Change": sensor["properties"].get("dateOfLastStatusChange"),
          "Description":   sensor["description"],
          "Measurement Unit": f"{datastreams["unitOfMeasurement"]["name"]}({datastreams["unitOfMeasurement"]["symbol"]})", 
          "Phenomenon Time": datastreams["phenomenonTime"],
          "Result Time": datastreams["resultTime"],
          "Sensor Name": sensor_api["name"], 
          "Observations Link": datastreams["Observations@iot.navigationLink"],
          "Location": location["location"],
          "Coordinates": location["location"]["coordinates"],
          "Sensor Installed": historical_location["time"],
          "Elevation":   sensor["properties"].get("elevationNAVD88"),
          }
    return list(map(create_sensor_obj, api_response["value"]))

In [ ]:
#things_data is a list of dictionaries where each dictionary contains information about...
#...a given thing
things_data = get_sensor_datastreams()
things_data

[{'name': 'gt-envsense-005',
  'id': 2,
  'status': 'outOfService',
  'Inundation Types': 'coastal,fluvial',
  'Installation Type': 'bridge',
  'Date of Last Status Change': '2024-03-18',
  'desc': 'Hutchinson Island Water Level Monitor',
  'Datastreams link': 'https://api.sealevelsensors.org/v1.0/Things(2)/Datastreams',
  'Historical Locations link': 'https://api.sealevelsensors.org/v1.0/Things(2)/HistoricalLocations',
  'Locations link': 'https://api.sealevelsensors.org/v1.0/Things(2)/Locations',
  'elev': '4.711',
  'coords': [-81.084099, 32.085707]},
 {'name': 'gt-envsense-004',
  'id': 3,
  'status': 'active',
  'Inundation Types': 'coastal,fluvial',
  'Installation Type': 'dock',
  'Date of Last Status Change': '2024-03-18',
  'desc': 'Kilkenny Creek Water Level Monitor',
  'Datastreams link': 'https://api.sealevelsensors.org/v1.0/Things(3)/Datastreams',
  'Historical Locations link': 'https://api.sealevelsensors.org/v1.0/Things(3)/HistoricalLocations',
  'Locations link': 'https

In [ ]:
#Filter our things that are not water sensors
for dict in things_data:
    if not "Water Level" in dict.get('desc'):
        things_data.remove(dict)

In [1]:
df = pd.DataFrame(things_data)
df

NameError: name 'pd' is not defined